In [ ]:
# This will tell you your current IP address from google colab
!curl api.ipify.org

In [ ]:
# Install needed python packages
!pip install pymongo

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
# Download us-counties.csv
!curl -L "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv" > us-counties.csv

In [ ]:
# Connect to MongoDB Atlas (insert your own credentials)
# Replace the placeholder with your Atlas connection string
uri = "<connection string>" # <-- put your own connection string here!

# Set the Stable API version when creating a new client
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
# Create or get your DB
db_name = "CS452_Mongo_Covid"
db = client.get_database(db_name)

Spark SQL Rewrite in MongoDB 1-6

*Redo the SparkSQL assignment in MongoDB using the aggregation pipeline.*

In [ ]:
# 1. Write code to define the schema and then read in the dataset
#    (took me 17 minutes!!!)

import json
import csv
import pandas
# Load the CSV file
df = pandas.read_csv('./us-counties.csv')
data = df.to_dict('records')
db.casesdeaths.drop()
db.casesdeaths.insert_many(data)
print("done")


In [ ]:
# 2. Write code to find the county with the most deaths

res = db.casesdeaths.find({},{"_id":0, "state":1, "county":1, "deaths":1}).sort({"deaths":-1}).limit(1)
list(res)

In [ ]:
# 3. Write code to find the county with the most cases


In [ ]:
# 4. Write code to find the total number of deaths in Utah county

In [ ]:
# 5. Write code to find the death rate for each state and sort the states by death rate descending

In [ ]:
# 6. Write code to something else interesting with this data – your choice

In this next part we will get experience using MongoDB's aggegregation pipeline's $lookup stage to join collections in MongoDb. Specifically we'll join to our **cases/deaths data** with **[vaccination data](https://ourworldindata.org/us-states-vaccinations#what-share-of-the-population-has-completed-the-initial-vaccination-protocol)** and **[total population data](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv)**.  First we need to download and ingest the data.



In [ ]:
# Get the CSV for covid vaccination data
!curl -L "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv" > "./us_state_vaccinations.csv"

In [ ]:
# Put the vaccinations data into the the DB  (took me 37 seconds)
with open("./us_state_vaccinations.csv") as f:
  dataRows = csv.DictReader(f)
  db.vaccinations.insert_many(dataRows)

df = pandas.read_csv('./us_state_vaccinations.csv')
data = df.to_dict('records')
db.vaccinations.drop()
db.vaccinations.insert_many(data)
print("Done!")


Use the \$lookup stage of the aggregation pipeline to join the vaccination and cases/deaths collections (you will need to group the covid case data by state) and use the \$out stage to create a new joined collection.

In [ ]:
# Create a "joined collection" of data by state for cases, deaths, vaccinations counts
# The cases and deaths data has over 2 million rows. You'll need to summarize it to
# states before you can successfully join it to the other dataset.

# YOUR CODE HERE




In [ ]:
# Get the total population (Use POPESTIMATE2019)
!curl -L "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv" > "./co-est2019-alldata.csv"

In [ ]:
# Put population data into the DB (took me 10 seconds)
# with open("./co-est2019-alldata.csv", encoding='latin-1') as f:
#   dataRows = csv.DictReader(f)
#   db.population.insert_many(dataRows)

df = pandas.read_csv('./co-est2019-alldata.csv', encoding='latin-1')
data = df.to_dict('records')
db.populations.drop()
db.populations.insert_many(data)
print("Done!")

Now let's incorrectly assume that vaccinations remove a person from possibility of infection and death. So based on our incorrect assumption, subtract the number who completed the initial vaccination protocol from the population of the state (POPESTIMATE2019 field) and and calculate a case percent and death percent of the population that was not ever vaccinated. 🤔

In [ ]:
# How will join the population data into this?



R1 - Report the highest 10 states and the lowest 10 states ordered by deaths divided by cases. (without subtracting vaccinated - see #5 from the SparkSQL redo)

In [ ]:
# Highest 10 states Deaths/Cases (death rate)


In [ ]:
# Lowest 10 states Deaths/Cases (death rate)


R2 - Report the highest 10 states and lowest 10 ordered by these new case and death percentages. (with subtracting vaccinated)

In [ ]:
# Highest 10 states (cases / (population - vaccinated))

In [ ]:
# Lowest 10 states (cases / (population - vaccinated))

In [ ]:
# Highest 10 states (deaths / (population - vaccinated))

In [ ]:
# Lowest 10 states (deaths / (population - vaccinated))